In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy
import os
import cv2
from PIL import Image,ImageDraw

In [2]:
df = pd.read_csv("C:/Users/ADMIN/OneDrive - Hanoi University of Science and Technology/Tài liệu/Projects/flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt", sep='\s+',header=None)

In [3]:
brand_map = {
    1: 'Ferrari',
    2: 'Ford',
    3: 'Nbc',
    4: 'Starbucks',
    5: 'RedBull',
    6: 'Mini',
    7: 'Unicef',
    8: 'Yahoo',
    9: 'Sprite',
    10: 'Texaco',
    11: 'Intel',
    12: 'Cocacola',
    13: 'Citroen',
    14: 'Heineken',
    15: 'Apple',
    16: 'Google',
    17: 'Fedex',
    18: 'Pepsi',
    19: 'Puma',
    20: 'DHL',
    21: 'Porsche',
    22: 'Nike',
    23: 'Vodafone',
    24: 'BMW',
    25: 'McDonalds',
    26: 'HP',
    27: 'Adidas',
}

In [4]:
def get_id_by_brand(name):
    for id, brand in brand_map.items():
        if brand == name:
            return id
    # If the name is not found, return None
    return None


In [5]:
print(df)

                   0       1  2    3    4    5    6
0      144503924.jpg  Adidas  1   38   12  234  142
1     2451569770.jpg  Adidas  1  242  208  413  331
2      390321909.jpg  Adidas  1   13    5   89   60
3     4761260517.jpg  Adidas  1   43  122  358  354
4     4763210295.jpg  Adidas  1   83   63  130   93
...              ...     ... ..  ...  ...  ...  ...
4531  2126991906.jpg   Yahoo  6   15    6  253   54
4532   217288720.jpg   Yahoo  6  136  161  304  222
4533  2472817996.jpg   Yahoo  6    2    4  499  106
4534  2514220918.jpg   Yahoo  6    1   69  342  157
4535   386891249.jpg   Yahoo  6  156   10  310   49

[4536 rows x 7 columns]


In [6]:
datadir = 'C:/Users/ADMIN/OneDrive - Hanoi University of Science and Technology/Tài liệu/Projects/flickr_logos_27_dataset/flickr_logos_27_dataset_images'

In [7]:
from PIL import Image, ImageDraw
import os
import sys

def draw_box(datadir, annotations):
    grouped_annotations = pd.DataFrame(annotations).groupby(0)
    for image_path, group in grouped_annotations:
            image_path = os.path.join(datadir, image_path).replace("\\", "/")
            with Image.open(image_path) as img:
                # Create a drawing context
                draw = ImageDraw.Draw(img)
                for i in range(len(group)):
                    x_min = group.iloc[i, 3]
                    y_min = group.iloc[i, 4]
                    x_max = group.iloc[i, 5]
                    y_max = group.iloc[i, 6]
                    
                    
                    # Normalize bounding box coordinates
                    x_min = float(x_min)
                    y_min = float(y_min)
                    x_max = float(x_max)
                    y_max = float(y_max)

                    # Draw the bounding box on the image
                    draw.rectangle(((x_min, y_min), (x_max, y_max)), outline="green", width=2)

                    # Add text to the bounding box
                    class_id = get_id_by_brand(group.iloc[i, 1])
                    class_name = group.iloc[i, 1]
                    text_size = draw.textsize(class_name)
                    draw.rectangle(((x_min, y_min - text_size[1]), (x_min + text_size[0], y_min)), fill="green")
                    draw.text((x_min, y_min - text_size[1]), class_name, fill="white")
                    
                # Display image with bounding boxes and class labels
                img.show()


In [8]:
#draw_box(datadir,df)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19356\3542251908.py:39: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(class_name)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19356\3542251908.py:39: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(class_name)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19356\3542251908.py:39: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(class_name)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19356\3542251908.py:39: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(class_name)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19356\3542251908.py:

KeyboardInterrupt: 

In [ ]:
#Loại bỏ các annonation trùng nhau
result = df.drop_duplicates(subset=[0, 1, 3, 4, 5, 6], keep='first')
print(result)

In [1]:
import io
from object_detection.utils import dataset_util
import string

def create_tf_examples(datadir:string, annotations:string, output_file:string):
    """
    Tạo input cho model
    Parameter
    ----------------
    - datadir: đường dẫn đến các ảnh 
    - annonation: đường dẫn đến file chứa thông tin ảnh(tên ảnh, brand, vị trí của brand trong ảnh)
    - ouput_file: đường dẫn của outputfile

    Returns
    ----------------
    - example.tfrecord
        + File chứa các thông tin để train
        + Lưu vào đường dẫn output_file  
    """
    # Split data into train and test sets
    grouped_annotations = pd.DataFrame(annotations).groupby(0)
    with tf.io.TFRecordWriter(output_file) as writer:
        for image_path, group in grouped_annotations:
            image_path = os.path.join(datadir, image_path).replace("\\", "/")
            with tf.io.gfile.GFile(image_path, 'rb') as f:
                encoded_image = f.read() 

            image = Image.open(image_path)
            width, height = image.size

            class_texts = []
            x_min_norms = []
            y_min_norms = []
            x_max_norms = []
            y_max_norms = []
            class_ids = []
            image_format = b'jpg'

            for i in range(len(group)):
                x_min = group.iloc[i, 3]
                y_min = group.iloc[i, 4]
                x_max = group.iloc[i, 5]
                y_max = group.iloc[i, 6]
                filename = group.iloc[i,0].encode('utf8')

                # Normalize bounding box coordinates
                x_min = float(x_min)
                y_min = float(y_min)
                x_max = float(x_max)
                y_max = float(y_max)

                x_min_norm = x_min / width
                y_min_norm = y_min / height
                x_max_norm = x_max / width
                y_max_norm = y_max / height
                class_id = get_id_by_brand(group.iloc[i, 1])
                class_text = group.iloc[i, 1]
                
                x_min_norms.append(x_min_norm)
                y_min_norms.append(y_min_norm)
                x_max_norms.append(x_max_norm)
                y_max_norms.append(y_max_norm)
                class_ids.append(class_id)
                class_texts.append(class_text.encode("utf8"))
            
            """
            Mã hóa thông tin để làm input cho model
            - Chiều cao ảnh
            - Chiều rộng ảnh
            - Tên ảnh
            - ID ảnh
            - Ảnh
            - Format(jpg)
            - xmin,ymin(tọa độ góc dưới trái của bounding box)
            - xmax,ymax(tọa độ góc trên trái của boudning box)
            - class/text: tên class
            - class/label: id của class
            """
            tf_example = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(height),                                                    
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(filename),
                'image/encoded': dataset_util.bytes_feature(encoded_image),
                'image/format': dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin': dataset_util.float_list_feature(x_min_norms),
                'image/object/bbox/xmax': dataset_util.float_list_feature(x_max_norms),
                'image/object/bbox/ymin': dataset_util.float_list_feature(y_min_norms),
                'image/object/bbox/ymax': dataset_util.float_list_feature(y_max_norms),
                'image/object/class/text': dataset_util.bytes_list_feature(class_texts),
                'image/object/class/label': dataset_util.int64_list_feature(class_ids),
                }))
            writer.write(tf_example.SerializeToString())

KeyboardInterrupt: 

In [ ]:
tfrecord_path = 'C:/Users/ADMIN/Documents/TensorFlow/workspace/training_demo/annotations/train.tfrecord'

In [11]:
create_tf_examples(datadir,result,tfrecord_path)

In [15]:
#Đọc file tfreccord đã tạo 
raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
raw_dataset
for raw_record in raw_dataset.take(6):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\002\000\034\000\034\000\000\377\376\000\027Created with The GIMP\377\333\000C\000\001\001\001\001\001\001\001\001\001\001\001\001\001\002\002\003\002\002\002\002\002\004\003\003\002\003\005\004\005\005\005\004\004\004\005\006\007\006\005\005\007\006\004\004\006\t\006\007\010\010\010\010\010\005\006\t\n\t\010\n\007\010\010\010\377\333\000C\001\001\001\001\002\002\002\004\002\002\004\010\005\004\005\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\010\377\300\000\021\010\000B\000\264\003\001\021\000\002\021\001\003\021\001\377\304\000\035\000\000\001\005\001\001\001\001\000\000\000\000\000\000\000\000\000\006\005\007\010\t\n\013\004\003\002\377\304\000F\020\000\001\003\003\003\003\002\003\006\002\005\007\r\000